## Final Project: Machine Learning Principles

### Instructions

### Supervised Learning

#### Import and Process Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/My Drive/MLP/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv")

### Data Link https://www.unb.ca/cic/datasets/ids-2018.html

First we inspect the dataframe to see what our features are.

In [ ]:
df

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773.0,202,0,61.444444,87.534438,1460,0,539.000000,655.432936,30597.305230,113.166177,9425.666667,19069.116850,73403.0,1.0,141385.0,17673.1250,23965.32327,73403.0,22.0,51417.0,8569.50000,13036.89082,31525.0,1.0,0,0,0,0,192,152,63.655975,49.510203,0,1460,254.470588,474.712955,225352.389700,0,0,1,1,0,0,0,1,0,270.375000,61.444444,539.000000,0,0,0,0,0,0,9,553,7,3773,8192,119,4,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0.0,38,0,19.000000,26.870058,0,0,0.000000,0.000000,135231.316700,10676.156580,140.500000,174.655375,264.0,17.0,281.0,281.0000,0.00000,281.0,281.0,0.0,0.00000,0.00000,0.0,0.0,1,0,0,0,40,20,7117.437722,3558.718861,0,38,19.000000,21.939310,481.333333,0,1,0,0,1,0,0,0,0,25.333333,19.000000,0.000000,0,0,0,0,0,0,2,38,1,0,123,0,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527.0,385,0,98.727273,129.392497,1460,0,701.800000,636.314186,41501.086400,92.915547,11192.960000,24379.448340,112589.0,1.0,279728.0,27972.8000,36167.74032,112589.0,94.0,258924.0,18494.57143,36356.50372,133669.0,1.0,0,0,0,0,232,312,39.310424,53.605123,0,1460,430.111111,566.234209,320621.179500,0,0,1,1,0,0,0,1,1,446.653846,98.727273,701.800000,0,0,0,0,0,0,11,1086,15,10527,8192,1047,5,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,02/03/2018 08:47:40,132,2,0,0,0.0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,15151.515150,132.000000,0.000000,132.0,132.0,132.0,132.0000,0.00000,132.0,132.0,0.0,0.00000,0.00000,0.0,0.0,0,0,0,0,40,0,15151.515150,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,0,0,256,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141.0,517,0,142.777778,183.887722,1460,0,472.384615,611.180489,27100.607260,80.287282,13048.380950,26311.627030,114077.0,1.0,273946.0,34243.2500,37996.56546,114077.0,201.0,252994.0,21082.83333,39075.73819,135611.0,1.0,0,0,0,0,192,272,32.844797,47.442485,0,1460,322.869565,497.254764,247262.300400,0,0,1,1,0,0,0,1,1,337.545455,142.777778,472.384615,0,0,0,0,0,0,9,1285,13,6141,8192,1047,5,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,3389,6,02/03/2018 02:08:18,3982183,14,8,1442,1731.0,725,0,103.000000,191.125412,1179,0,216.375000,405.464791,796.799143,5.524608,189627.761900,257112.296400,953396.0,34.0,3982183.0,306321.7692,340816.37320,1221092.0,23468.0,3930879.0,561554.14290,350274.78920,953825.0,131451.0,0,0,0,0,292,172,3.515660,2.008948,0,1179,137.956522,278.879491,77773.770750,0,0,1,1,0,0,0,1,0,144.2

We notice we have two labels to work with Benign and Bot, so our classifier algorithm will be trying to determine these two labels.

In [ ]:
df['Label'].unique()

array(['Benign', 'Bot'], dtype=object)

#### Build and Test Models

In [ ]:
X = df.drop(['Label','Timestamp'], axis=1)
X = X.replace([np.inf, -np.inf, np.nan], 0)
X = X.reset_index()
y = df['Label']
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

def test_accuracy(model,X_train, X_test, y_train, y_test):
  predict_Y = model.predict(X_train)
  pre_vs_act = predict_Y == y_train
  train_acc = len(pre_vs_act[pre_vs_act == True]) / len(pre_vs_act)  

  predict_Y = model.predict(X_test)
  pre_vs_act = predict_Y == y_test
  test_acc = len(pre_vs_act[pre_vs_act == True]) / len(pre_vs_act)
  
  return train_acc, test_acc

def model_test(model):
  rs = ShuffleSplit(n_splits=10, test_size=.5, random_state=42)
  train_acc_list = []
  test_acc_list = []
  for train_ind, test_ind in rs.split(X):
    X_train = X.iloc[train_ind,:]
    X_test = X.iloc[test_ind,:]
    y_train = y[train_ind]
    y_test = y[test_ind]
    # to go back le.inverse_transform
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    model.fit(X_train,y_train)
    train_acc, test_acc = test_accuracy(model,X_train, X_test, y_train, y_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
  print("Train Accuracy is: " + str(np.asarray(train_acc_list).mean()))
  print("Test Accuracy is: " + str(np.asarray(train_acc_list).mean()))


In [ ]:
  #Base Accuracy is assign most common label. Essentiall assign everything as benign.
print(f"Base Accuracy is {len(y[np.where(y == 0)]) / len(y)}")

Base Accuracy is 0.7270667334239325


In [ ]:
# Linear SVC Did Not Finish
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
#C = 1.0  # SVM regularization parameter
#model = svm.LinearSVC(C=C, max_iter=100000)
#model.fit(X_train,y_train)
#test_accuracy(model, X_train, X_test, y_train, y_test)

In [ ]:
# SGD with no normalization, l2 regularization
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
model = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42, penalty='l1',class_weight='balanced')
model.fit(X_train,y_train)
test_accuracy(model, X_train, X_test, y_train, y_test)

(0.9240702134517926, 0.923919677734375)

In [ ]:
# SGD with normalization, l2 regularization
model = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
model_test(model)

Train Accuracy is: 0.996557229151209
Test Accuracy is: 0.996557229151209


In [ ]:
# SGD with l1 regularization
model = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42, penalty='l1')
model_test(model)

Train Accuracy is: 0.9974176357605662
Test Accuracy is: 0.9974176357605662


In [ ]:
#SGD with squared_hinge loss function. Finishes, but too long of runtime and worse performing
#model = SGDClassifier(loss='squared_hinge', max_iter=1000, tol=1e-3, random_state=42, penalty='l1',)
#model_test(model)

In [ ]:
#BEST SGD with l1 regularization and class_weight to control for unbalanced classes
model = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42, penalty='l1',class_weight='balanced')
model_test(model)

Train Accuracy is: 0.9974807691207296
Test Accuracy is: 0.9974807691207296


In [ ]:
#SGD with elasctinet regularization, class weighting
model = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42, penalty='elasticnet',class_weight='balanced')
model_test(model)

Train Accuracy is: 0.9960657044710244
Test Accuracy is: 0.9960657044710244


In [ ]:
# SGD with l1, class_weight, and bigger max_iter
model = SGDClassifier(max_iter=10000, tol=1e-3, random_state=42, penalty='l1',class_weight='balanced')
model_test(model)

Train Accuracy is: 0.9974807691207296
Test Accuracy is: 0.9974807691207296


Additional attempts:
1. Default LinearSVC with 1.0 regularization parameter, no normalization. Model did not train within 25 mins for 100,000 max iterations.
2. Default LinearSVC with 1.0 regularization parameter, no normalization. Model training finished in ~ 3 mins without converging for 100 max iterations.
3. Default SGD classifier with 1000 iter and tol=1e-3 finished in seconds. Not great results. Train .98 Test .73
4. Tried to increase max_iter up to 1000000, but still not much improvement on test

### Unsupervised Learning

#### Import Data

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

### Data Link https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/

In [ ]:
columns = ["srcip","sport","dstip","dsport","proto","state","dur","sbytes","dbytes","sttl","dttl","sloss","dloss","service","Sload","Dload","Spkts","Dpkts","swin","dwin","stcpb","dtcpb","smeansz","dmeansz","trans_depth","res_bdy_len","Sjit","Djit","Stime","Ltime","Sintpkt","Dintpkt","tcprtt","synack","ackdat","is_sm_ips_ports","ct_state_ttl","ct_flw_http_mthd","is_ftp_login","ct_ftp_cmd","ct_srv_src","ct_srv_dst","ct_dst_ltm","ct_src_ ltm","ct_src_dport_ltm","ct_dst_sport_ltm","ct_dst_src_ltm","attack_cat","Label"]
df = pd.read_csv("/content/drive/My Drive/MLP/UNSW-NB15_1.csv", names=columns)
df = df[df['attack_cat'].notna()]
df = df.drop(columns="Label")
df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat
20,175.45.176.3,21223,149.171.126.18,32780,udp,INT,0.000021,728,0,254,0,0,0,-,1.386667e+08,0.000000,2,0,0,0,0,0,364,0,0,0,0.000000,0.000000,1421927415,1421927415,0.021000,0.000000,0.000000,0.000000,0.000000,0,2,0,0,0,1,1,1,1,1,1,1,Exploits
21,175.45.176.2,23357,149.171.126.16,80,tcp,FIN,0.240139,918,25552,62,252,2,10,http,2.805042e+04,815794.187500,12,24,255,255,1708297952,1939490744,77,1065,1,12026,1170.481668,1144.383360,1421927416,1421927416,21.830818,9.570304,0.051475,0.006528,0.044947,0,1,1,0,0,3,2,2,1,1,1,1,Exploits
22,175.45.176.0,13284,149.171.126.16,80,tcp,FIN,2.390390,1362,268,254,252,6,1,http,4.233619e+03,749.668518,14,6,255,255,3897219059,2466816006,97,45,1,0,18786.711400,941.724938,1421927414,1421927416,183.579303,474.259406,0.066088,0.017959,0.048129,0,1,1,0,0,5,2,2,1,1,1,1,Reconnaissance
39,175.45.176.2,13792,149.171.126.16,5555,tcp,FIN,0.175190,8168,268,254,252,4,1,-,3.463668e+05,10228.894530,14,6,255,255,2505143795,3592239707,583,45,0,0,774.788316,47.765387,1421927417,1421927417,11.837692,33.287000,0.054878,0.008744,0.046134,0,1,0,0,0,1,1,1,1,1,1,1,Exploits
40,175.45.176.2,26939,149.171.126.10,80,tcp,FIN,0.190600,844,268,254,252,2,1,http,3.189927e+04,9401.888672,10,6,255,255,3006332195,1452987536,84,45,1,0,996.632407,59.532129,1421927418,1421927418,18.573778,36.845602,0.050675,0.006354,0.044321,0,1,1,0,0,3,1,1,1,1,1,1,Exploits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186498,175.45.176.1,58463,149.171.126.15,179,tcp,FIN,31.461020,924,708,254,252,2,1,-,2.204633e+02,168.843857,16,16,255,255,2042981190,1213459295,58,44,0,0,249824.254900,7606.752000,1421934405,1421934437,2097.401667,2092.153875,0.111803,0.060145,0.051658,0,1,0,0,0,6,6,2,2,2,1,2,Fuzzers
186499,175.45.176.1,58460,149.171.126.15,179,tcp,FIN,57.995934,1362,536,254,252,6,1,-,1.784953e+02,67.866829,20,12,255,255,3745930258,3164328572,68,45,0,0,414392.372000,16429.892000,1421934380,1421934438,3052.417579,5263.194000,0.144543,0.082349,0.062194,0,1,0,0,0,6,6,2,2,2,1,2,Fuzzers
186562,175.45.176.1,58967,149.171.126.15,179,tcp,FIN,32.361782,1128,622,254,252,6,1,-,2.635207e+02,142.884598,18,14,255,255,3446610601,3189881313,63,44,0,0,241558.467600,8392.374000,1421934407,1421934439,1903.634060,2483.182250,0.062474,0.062474,0.000000,0,1,0,0,0,6,6,2,2,2,1,2,Fuzzers
186658,175.45.176.1,58485,149.171.126.15,179,tcp,FIN,0.476639,470,354,254,252,1,1,-,5.203099e+03,6915.086426,8,8,255,255,620124694,2915203193,59,44,0,0,108.230812,110.224789,1421934442,1421934443,60.043285,66.460289,0.000000,0.000000,0.000000,0,1,0,0,0,8,8,2,2,2,1,2,Fuzzers


In [ ]:
df['attack_cat'].unique()

array(['Exploits', 'Reconnaissance', 'DoS', 'Generic', 'Shellcode',
       ' Fuzzers', 'Worms', 'Backdoors', 'Analysis'], dtype=object)

In [ ]:
y = df['attack_cat']
X = df.drop(columns=["attack_cat","srcip","sport","dstip","dsport","proto","state","service"])

#### Build and Test Models

In [ ]:
def test_clustering(model, X_train, X_test, y_train, y_test):
  print("Training Data")
  labels = kmeans.predict(X_train)
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, y_train))
  print("Completeness: %0.3f" % metrics.completeness_score(labels, y_train))
  print("V-measure: %0.3f" % metrics.v_measure_score(labels, y_train))

  print("\nTesting Data")
  labels = kmeans.predict(X_test)
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, y_test))
  print("Completeness: %0.3f" % metrics.completeness_score(labels, y_test))
  print("V-measure: %0.3f" % metrics.v_measure_score(labels, y_test))

In [ ]:
# KMeans without normaliztion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
kmeans = KMeans(n_clusters=9, random_state=42).fit(X_train)
test_clustering(kmeans, X_train, X_test, y_train, y_test)

Training Data
Homogeneity: 0.135
Completeness: 0.128
V-measure: 0.131

Testing Data
Homogeneity: 0.129
Completeness: 0.119
V-measure: 0.124


In [ ]:
def test_clustering(X, Y):
  rs = 42
  train_h_l = []
  train_c_l = []
  train_v_l = []
  test_h_l = []
  test_c_l = []
  test_v_l = []
  for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=rs)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    kmeans = KMeans(n_clusters=9, random_state=rs).fit(X_train)

    labels = kmeans.predict(X_train)
    train_h_l.append(metrics.homogeneity_score(labels, y_train))
    train_c_l.append(metrics.completeness_score(labels, y_train))
    train_v_l.append(metrics.v_measure_score(labels, y_train))

    labels = kmeans.predict(X_test)
    test_h_l.append(metrics.homogeneity_score(labels, y_test))
    test_c_l.append(metrics.completeness_score(labels, y_test))
    test_v_l.append(metrics.v_measure_score(labels, y_test))
    rs += 1

  print("Training Data")
  labels = kmeans.predict(X_train)
  print("Homogeneity: %0.3f" % np.asarray(train_h_l).mean())
  print("Completeness: %0.3f" % np.asarray(train_c_l).mean())
  print("V-measure: %0.3f" % np.asarray(train_v_l).mean())

  print("\nTesting Data")
  labels = kmeans.predict(X_test)
  print("Homogeneity: %0.3f" % np.asarray(test_h_l).mean())
  print("Completeness: %0.3f" % np.asarray(test_c_l).mean())
  print("V-measure: %0.3f" % np.asarray(test_v_l).mean())    

In [ ]:
# Kmeans with normalization
test_clustering(X,y)

Training Data
Homogeneity: 0.205
Completeness: 0.182
V-measure: 0.193

Testing Data
Homogeneity: 0.206
Completeness: 0.183
V-measure: 0.194


In [ ]:
# Kmeans, but now X is adding in categorical features previously omitted
X = df.drop(columns=["attack_cat"])
categories = ["proto","state","service", "sport", "srcip","dstip"]
for cat in categories:
  X[cat] = X[cat].astype('category')
  X[cat] = X[cat].cat.codes
test_clustering(X,y)

Training Data
Homogeneity: 0.220
Completeness: 0.200
V-measure: 0.210

Testing Data
Homogeneity: 0.221
Completeness: 0.201
V-measure: 0.210


In [ ]:
# Kmeans but now categories are one hot encoded instead of simply normalized
X = df.drop(columns=["attack_cat"])
X.reset_index(inplace=True)
categories = ["proto","state","service", "sport", "srcip","dstip"]
for cat in categories:
  X[cat] = X[cat].astype('category')
  X[cat] = X[cat].cat.codes
  enc = OneHotEncoder(handle_unknown='ignore')
  enc_df = pd.DataFrame(enc.fit_transform(X[['proto']]).toarray())
  enc_df.columns = [str(col) + cat for col in enc_df.columns]
  X = X.join(enc_df)
test_clustering(X,y)

Training Data
Homogeneity: 0.345
Completeness: 0.212
V-measure: 0.262

Testing Data
Homogeneity: 0.351
Completeness: 0.213
V-measure: 0.265


## Report

### Supervised Learning

In this project, I used the supervised learning algorithm known as a Support Vector Machine (SVM) to solve a classification problem. The problem is to classify network traffic as either being benign or belonging to a botnet.

### Dataset
The main data set I will use to train and verify my models will be the CSE-CIC-IDS2018 dataset [1]. The Canadian Institute for Cybersecurity (CIC) built a mock network including approximately 500 machines using a typical network architecture [1]. The attacking infrastructure utilizes seven different attack types on the target network [1]. The data includes approximately 220 GiB of full packet capture and session feature-statistics generated by the CICFlowMeter tool [1]. Some examples of the 80 session statistic-features include flow duration, average size of packet, flow size, destination port, etc. [1]. I use the session statistic-features to construct my SVM. Each data point is labeled as benign or bot, and these serve as my labels for supervised learning. For this model I use a set of data from one specific day where they were testing the Zues and Ares botnets. An interesting feature of this dataset is that it is unbalanced, meaning there are more data points for the benign class than there are for the bot class.

### Algorithm
I choose the stochastic gradient descent SVM model from Scikit-learn, SGDClassifier [2], because I needed a classification algorithm that could handle large datasets. My data set has 1048575 rows × 80 columns. I first tried to use the LinearSVC model, but this was unable to complete training in a reasonable time limit (I waited 25mins before stopping). In comparison, the stochastic gradient descent version of the algorithm completed in less than a minute for one run.

I did an exploration of the model parameters to determine which parameters resulted in the highest model accuracy.


### Cross Validation
For this model, I used the cross-validation technique of Montecarlo validation. In this method, I select training and testing data randomly for a total of 10 iterations. The greater the number of iterations, the more confident we can be in our results. However, due to the large data size, the model takes a significant amount of time to train. Thus, I limited the number of iterations to 10.

In addition to using cross validation to have higher confidence in my accuracy measurements, I computed a baseline accuracy to compare the models to. This baseline is simply applying the most common label to the data set. I found this baseline accuracy to be ~72.7%.


### Regularization
For this model, I tried three forms of regularization:
*	Lasso Regularization or L1
*	Ridge Regularization or L2
*	Elasticnet Regularization which is a linear combination of L1 and L2.

Regularization is used to add a penalty to the error function that punishes weights that become too large. I found that for this model the L1 regularization performed the best, albeit a very modest improvement of model accuracy (.009% when modifying the default L2 to be L1, elasticnet performed even worse than the default L2.)


### Normalization
I tested this model with normalized data and the original data to measure how important data normalization is for SVM training. SVM assumes data inputs are normalized, meaning the data should have zero mean and unit variance. I determined there was approximately 7% improvement when comparing unnormalized data and normalized data (~92 % accurate unnormalized, ~99% accurate normalized). I was surprised to see the unnormalized data perform as well as it did, and glad to see a significant improvement when taking the time to normalize the data.

### Loss Function
The SVM model can be trained with multiple loss functions. I tested the default loss function, the hinge function, versus the squared hinge loss function. I found the squared hinge loss function took significantly longer time to train, and did not have as good as an accuracy.

### Class Weighting
I also tested using class weighting because my I have an unbalanced dataset. I found a small improvement when using this option. When using class weight, data samples from small classes are given more weight when training the model to counteract the bias for the dominant class.

### Training iterations
I found increasing the max number of allowable training iterations did not increase accuracy. This is because my model was converging before reaching the max iteration.

### Accuracy
Overall this model was extremely accurate with a final accuracy of 99.75% accuracy for a model that used the default SGDClassifier and changing to L1 regularization, using class weighting, and ensuring to normalize the input data.

## Unsupervised Learning
Next, I used the unsupervised learning algorithm known as KMeans to solve a clustering problem. Sticking with the network analysis theme, I found a second dataset that contained nine classes of network attacks. After removing the benign traffic, my goal was to see if I could strongly cluster the nine separate attack classes.

### Dataset
For this model, I used the UNSW-NB15 dataset [3]. This dataset uses the Bro tool (now known as Zeek) to compute network session statistics, and is similar to the CICFlowMeter dataset used in the supervised example. However, this example contains a much larger variety of attack types. I choose this dataset for clustering to see if network traffic characteristics can be used to cluster varieties of malware (Exploits, Reconnaissance, DoS, Generic, Shellcode Fuzzers, Worms, Backdoors, and Analysis). This data had more categorical features than the first, so it took a significant amount of data cleaning to improve results.

### Algorithm
I chose to use KMeans because I needed a clustering algorithm that worked with unlabeled data and is able to generate a specified number of clusters. Despite having a large dataset size, 22215 rows × 48 columns, I was happy to find the built in Scikit-Learn KMeans implementation was able to perform well [4].

### Model Measurement
Because KMeans is working with unlabeled data to generate clusters, I choose three metrics that interpret how the clusters relate to each other. From these metrics, I can understand if the nine attack classes are clearly separable or if they have significant overlap. They are also convenient to use because Scikit-learn's implementation does not require you to map true labels to predicted labels, so they are independent of the absolute label values. The three metrics are:
*	Homogeneity which reports a value from [0 to 1] measuring if all its clusters contain only data points which are members of a single class.
*	Completeness which reports a value from [0 to 1] if data points that are members of a given class are elements of the same cluster.
*	V-measure reports a value from [0 to 1] that is the harmonic mean between homogeneity and completeness


### Cross Validation
For this model, I used the cross-validation technique of Montecarlo validation. In this method, I select training and testing data randomly for a total of 10 iterations. The greater the number of iterations, the more confident we can be in our results.

### Normalization
I tested this model with normalized data and the original data to measure how important data normalization is for SVM training. I determined that normalizing data is extremely important for the KMeans algorithm. Before normalizing I had scores (Homogeneity: 0.129, Completeness: 0.119, V-measure: 0.12) and after normalizing (Homogeneity: 0.206, Completeness: 0.183, V-measure: 0.194). This was one of the single largest model improvements I made.

### Categorical features
Initially I removed the categorical features because I assumed the model would not require them to be very accurate. As seen in my metric scores above, this was not the case. I added in the categorical features and improved my metrics to (Homogeneity: 0.221, Completeness: 0.201, V-measure: 0.210). This was a modest improvement, but underwhelming as the newly included features (proto, state, service, sport, srcip, dstip), include many of the features normally used to distinguish network traffic sessions. 

### One-Hot Encoding Categorical Features
My last attempt to improve accuracy included one-hot encoding the categorical features instead of normalizing them as if they were ordinal data. This resulted in another large improvement brining my metrics to (Homogeneity: 0.351, Completeness: 0.213, V-measure: 0.265)

### Results
As seen by our final metrics, (Homogeneity: 0.351, Completeness: 0.213, v-measure: 0.265), I determined that there is too much cluster overlap for this model to reliably cluster network traffic attack types. However, I determined that normalizing data and one-hot encoding categorical data provided significant improvements.

### References
[1] Intrusion Detection Evaluation Dataset (CIC-IDS2018). University of New Brunswick, Canadian Institute for Cybersecurity, www.unb.ca/cic/datasets/ids.html.

[2] https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

[3] Moustafa, Nour, and Jill Slay. "UNSW-NB15: a comprehensive data set for network intrusion detection systems (UNSW-NB15 network data set)." Military Communications and Information Systems Conference (MilCIS), 2015. IEEE, 2015.

[4] https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html 